## Import the library.

In [19]:
import os
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import TokenTextSplitter
from langchain_community.vectorstores.azuresearch import AzureSearch
from langchain_community.retrievers import AzureAISearchRetriever
from langchain_openai import AzureOpenAIEmbeddings, OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters.character import CharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain_openai import AzureChatOpenAI

## API Details

In [21]:
os.environ["AZURE_AI_SEARCH_SERVICE_NAME"] = "YOUR  Azure_Ai_Search CONNECTION STRING"
os.environ["AZURE_AI_SEARCH_INDEX_NAME"] = "rag_vector1"
os.environ["AZURE_AI_SEARCH_API_KEY"] = "Your Api Key"
os.environ["AZURE_OPENAI_API_KEY"] = "Your Azure openai  Api key"
os.environ["AZURE_OPENAI_ENDPOINT"] = "Your Azure openai endpoint"
os.environ["OPENAI_API_VERSION"] = "Your Api version"

## Azure OpenAI Embeddings

In [22]:

azure_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_openai_api_key = os.environ["AZURE_OPENAI_API_KEY"]
azure_openai_api_version = os.environ["OPENAI_API_VERSION"]
azure_deployment = "Embedding-ada"

In [7]:
embeddings = AzureOpenAIEmbeddings(
    model=azure_deployment,
    azure_endpoint=azure_endpoint,
    openai_api_key=azure_openai_api_key,
)

## Createing Vector_store using same : AZURE_AI_SEARCH_INDEX_NAME

In [10]:
vector_store: AzureSearch = AzureSearch(
    embedding_function=embeddings.embed_query,
    azure_search_endpoint=os.getenv("AZURE_AI_SEARCH_SERVICE_NAME"),
    azure_search_key=os.getenv("AZURE_AI_SEARCH_API_KEY"),
    index_name="rag_vector1",
    search_type="similarity",)

name is not a known attribute of class <class 'azure.search.documents.indexes.models._index.SearchField'> and will be ignored
type is not a known attribute of class <class 'azure.search.documents.indexes.models._index.SearchField'> and will be ignored
key is not a known attribute of class <class 'azure.search.documents.indexes.models._index.SearchField'> and will be ignored
searchable is not a known attribute of class <class 'azure.search.documents.indexes.models._index.SearchField'> and will be ignored
filterable is not a known attribute of class <class 'azure.search.documents.indexes.models._index.SearchField'> and will be ignored
facetable is not a known attribute of class <class 'azure.search.documents.indexes.models._index.SearchField'> and will be ignored
sortable is not a known attribute of class <class 'azure.search.documents.indexes.models._index.SearchField'> and will be ignored
hidden is not a known attribute of class <class 'azure.search.documents.indexes.models._index.Sear

## Load and split documents

In [11]:

loader = TextLoader("D:/Azure_ai_search/beautiful_long_story.txt", encoding="utf-8")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

## Add documents to vector store

In [12]:

vector_store.add_documents(documents=docs)

['ZDUwNjEwYzAtZWYwMS00ZDk3LWIyOTUtMDJhYzBhNWQxNTM2',
 'NTg5NzA4YTYtYTEzNS00YzhkLTgzMTQtY2YxMTBiOWJmNmRj',
 'NmYyY2M1NzctZDAwMC00ZGFmLWE4MDMtNmVhNjMzODhiYTIx',
 'ZTU3ZWQyZGMtZDJmOS00NmJjLWI3YjEtNmI1NzFmZGY3NTYy',
 'YmY0NWMxZmQtMzI4Yi00MjFiLTllY2ItM2UzYWI4N2NjMzBl',
 'YTIyZjVmODEtMTIxYy00NDQ2LWI0YzItZTU4ODZhMjVlYjIw',
 'OTExYzc4NDItZWJiMS00YzlmLWFiNmMtNDk0OWE4ZGJkMDEy']

## Perform a similarity search

In [14]:

query = "tell me about Elmsworth"
similar_docs = vector_store.similarity_search(
    query=query,
    k=3,
    search_type="similarity",
)
print(similar_docs[0].page_content)

Once upon a time in a land far beyond the known realms, nestled between towering mountains and an endless sea, there lay the enchanting Kingdom of Elmsworth. The kingdom was renowned for its lush green landscapes, sparkling rivers, and the grandeur of its ancient castles. Yet, what truly set Elmsworth apart was its rich tapestry of myths and legends, whispered through the generations.

At the heart of Elmsworth stood the majestic Castle of Arion, where King Cedric, a wise and noble ruler, had reigned for over three decades. The king was beloved by his people for his fairness and his deep connection with the land. He was often seen wandering the forests, speaking with the animals, and tending to the ancient trees that had stood for centuries.


## Perform a hybrid search

In [15]:

hybrid_docs = vector_store.similarity_search(
    query="tell me about Elmsworth",
    k=3,
    search_type="hybrid",
)
print(hybrid_docs[0].page_content)

Once upon a time in a land far beyond the known realms, nestled between towering mountains and an endless sea, there lay the enchanting Kingdom of Elmsworth. The kingdom was renowned for its lush green landscapes, sparkling rivers, and the grandeur of its ancient castles. Yet, what truly set Elmsworth apart was its rich tapestry of myths and legends, whispered through the generations.

At the heart of Elmsworth stood the majestic Castle of Arion, where King Cedric, a wise and noble ruler, had reigned for over three decades. The king was beloved by his people for his fairness and his deep connection with the land. He was often seen wandering the forests, speaking with the animals, and tending to the ancient trees that had stood for centuries.


## Using Retrieval QA

In [16]:

GPT_DEPLOYMENT_NAME = "gpt-4o"
llm = AzureChatOpenAI(
    azure_endpoint=azure_endpoint,
    openai_api_version=azure_openai_api_version,
    azure_deployment=GPT_DEPLOYMENT_NAME,
)

retriever = vector_store.as_retriever()

qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriever
)

## Query using QA chain

In [18]:

query = "ell me about Elmsworth"
result = qa({"query": query})
print(result)

{'query': 'ell me about Elmsworth', 'result': "Elmsworth is an enchanting kingdom nestled between towering mountains and an endless sea. It is renowned for its lush green landscapes, sparkling rivers, and the grandeur of its ancient castles. The heart of Elmsworth is the majestic Castle of Arion, where King Cedric, a wise and noble ruler, has reigned for over three decades. King Cedric is beloved by his people for his fairness and his deep connection with the land, often seen wandering the forests, speaking with the animals, and tending to the ancient trees that have stood for centuries.\n\nElmsworth is distinguished by its rich tapestry of myths and legends, which are whispered through the generations. It is a land where people live in harmony with nature and each other, ever vigilant and prepared for any challenges that may arise. The kingdom's legends of bravery and unity inspire its inhabitants and future generations.\n\nThe Kingdom of Elmsworth faced a significant threat from the 